# Testing

El testeo es el proceso de *evaluar y verificar* que el código funciona *como queremos que funcione*. Es decir,que cumpla los *requisitos especificados* (labor de QA) y que no contenga errores (labor de Dev). El testeo puede ser **manual o automatizado**. La forma más sencilla de realizarlo es con la keyword `assert`.

In [1]:
def suma(a: int | float, b: int | float) -> int | float:
    return a + b

Según el teorema de Hoare, de la forma **{Q} S {R}**, se puede conocer el estado interno de un sistema *S* de acuerdo a su **precondición Q** y a su **postcondición R**. Este teorema **no** es correcto, pero da una buena base de como realizar testeo **unitario**, es decir, de un componente aislado.

Se crea una precondición, y se especifica una postcondición. Si el test es correcto, se llegará a esa postcondición.

In [ ]:
# Con assert

assert suma(2, 3) == 5, "La suma de 2 y 3 debería ser 5"
assert suma(-1, 1) == 0, "La suma de -1 y 1 debería ser 0"

# Con unittest, de la libreria estandart

import unittest

class TestSuma(unittest.TestCase):
    def test_suma_positivos(self):
        self.assertEqual(suma(2, 3), 5)
    
    def test_suma_negativos(self):
        self.assertNotEqual(suma(-1, 1), 10)

# Con pytest, instalado desde pip

import sys
!{sys.executable} -m pip install pytest
import pytest

def test_suma_positivos():
    assert suma(2, 3) == 5

def test_suma_negativos():
    assert suma(-1, 1) != 10

Sin embargo, muchos otros tipos de tests. Los **tests de integración** se encargan de ver como los componentes se comunican entre sí. Los **tests end to end** intentan simular el uso que un usuario daría al software. Los **tests por propiedad** intentan variar el *estado interno S*, en vez de la precondición y la postcondición...

In [8]:
# Un test de propiedad

def resta(a: int | float, b: int | float) -> int | float:
    return a - b

numero = 1  # Estrategia (numerica) ---> hypothesis
assert suma(2, resta(numero, 2)) == numero  # numero + 2 - 2 = numero, sin importar el numero!

## Test Driven Development

El **Test Driven Decelopment** (TDD) es una práctica que consiste en hacer *antes el test del código, para despues ir haciendo el código de acuerdo a los tests*. Esta metodología se llama **Red-Green-Refactor**

In [ ]:
# Ejemplo con patito que haga quack

## Hacer una calculadora

# Intro a Pytest

Pytest es un paquete de testeo muy potente y utilizado. Permite crear tests con una sintaxis sencilla (aprovechando `assert`) que es compatible con las otras librerias de testeo (notablemente `doctest` y `unittest`). Se utiliza para realizar testeo unitario y de integración.

## Sintaxis básica

```python
class Test>nombre>:
    def test_<nombre>():
        assert <condicion>
        with pytest.raises(<tipo error>, match=<mensaje de error>):  # En caso de un raise
            <condicion de error>
```

La forma de ejecutar los tests en Pytest es `pytest` o, si no encuentra el lugar (se que puede configurar), `pytest <path>`, aunque se puede especificar un modo verboso con `pytest -v` o un modo 1-error `pytest --maxfail=1`.

In [10]:
class ListaFrutas:
    def __init__(self):
        self.lista = []
        
    def añadir_fruta(self, fruta: str):
        if isinstance(fruta, str):
            self.lista.append(fruta)
        else:
            raise TypeError("Eso no es una fruta")

def test_añadir_fruta():
    instancia = ListaFrutas()
    fruta = "manzana"
    instancia.añadir_fruta(fruta)
    assert instancia.lista[0] == fruta
    with pytest.raises(TypeError, match="Eso no es una fruta"):
        instancia.añadir_fruta(100)

ERROR: usage: ipykernel_launcher.py [options] [file_or_dir] [file_or_dir] [...]
ipykernel_launcher.py: error: unrecognized arguments: -f
  inifile: None
  rootdir: C:\Users\Proye



<ExitCode.USAGE_ERROR: 4>

## Setup y teardown

Pytest permite hacer *setup* como una forma de preparar un test, o una serie de tests; y *teardown* como forma de limpiar las condiciones una vez estos tests han terminado. El alcance de este *setup* y *teardown* puede ser a nivel de modulo y nivel de clase o función.

In [ ]:
# Mismo codigo que antes pero preparando la instancia con setup y eliminandola con teardown

def setup_ListaFrutas(function):
    global instancia  # Crea una instancia global para los tests
    instancia = ListaFrutas()
    print(f"\nConfigurando la prueba: {function.__name__}")

def teardown_ListaFrutas(function):
    global instancia
    del instancia  # Elimina la instancia
    print(f"Limpieza después de la prueba: {function.__name__}")

def test_añadir_fruta():
    fruta = "manzana"
    instancia.añadir_fruta(fruta)  # Ya se ha hecho setup, no es necesario instanciar
    assert instancia.lista[0] == fruta

## Fixtures

Sin embargo, Pytest utiliza **fixtures** para manejar el *setup* y el *teardown*. Los fixtures son simplemente funcines que se usan para configurar el estado previo a unos tests, inyectandose en las pruebas de forma automática.

In [ ]:
# Mismo codigo con fixture

@pytest.fixture
def instancia():
    return ListaFrutas()  # Se prepara la instancia

def test_añadir_fruta(instancia):  # Se inyecta la fixture
    fruta = "manzana"
    instancia.añadir_fruta(fruta)
    assert instancia.lista[0] == fruta

## Tests parametrizados

Los tests parametrizados son una forma de hacer una bateria de tests, inyectando múltiples precondiciones a unos tests particulares.

In [ ]:
@pytest.fixture
def instancia():
    return ListaFrutas()

@pytest.mark.parametrize("fruta, resultado", [  # Se crea una lista con la fruta y la fruta que se espera conseguir (en este caso la misma)
    ("manzana", "manzana"),
    ("pera", "pera"),
    ("plátano", "plátano")
])

def test_añadir_fruta_parametrizado(instancia, fruta, resultado):  # Se inyecta no solo la fixture sino tambien la parametrizacion
    instancia.añadir_fruta(fruta)  # Primera parte de la parametrizacion (fruta)
    assert instancia.lista[-1] == resultado  # Segunda parte de la parametrizacion (resultado)

## Marcas

Pytest permite marcar los tests de acuerdo a diferentes criterios, incluso crear nuestras propias marcas.

In [11]:
@pytest.mark.xfail
def test_sumar_falla():
    assert sumar(2, 2) == 5  # Esto fallara intencionadamente

In [ ]:
@pytest.mark.skipif(sys.version_info < (3, 8), reason="Requiere Python 3.8 o superior")
def test_funcion_python_3_8():
    assert True  # Este test se salta si la version de Python es menor de 3.8, y se ejecuta si se tiene esa version o superior

## Coverage

Pytest se puede extender con plugins. Uno de los más usados es *coverage*, que permite ver el **coverage** del código.
Se instala con `pip install pytest-cov` y mide el porcentaje de código que nuestros tests han cubierto.

Se utiliza como `pytest --cov=<paquete>`

# Principios SOLID

Los *principios SOLID* son un conjunto de cinco **principios de diseño** que promueven la creación de sistemas más *comprensibles, flexibles y mantenibles*. Se basan en cinco principios, que aseguran un código de calidad. Y como consecuencia de estos, *surjen los patrones de diseño*.

## Interfaces y ABC

Las **ABC (Abstract Base Classes)** son *clases base abstractas* definidas en el módulo abc que permiten definir *métodos abstractos*. Estos métodos **deben ser implementados por cualquier clase que herede de la ABC**, lo que garantiza que todas las **subclases cumplan con una interfaz específica**. Las ABCs son útiles para crear una *arquitectura sólida y coherente* en la que todas las subclases cumplen con ciertas expectativas en términos de métodos y comportamiento.

In [12]:
# Creando la interfaz

from abc import ABC, abstractmethod

class Vehiculo(ABC):

    @abstractmethod
    def arrancar(self):
        pass

    @abstractmethod
    def detener(self):
        pass

    def identificar(self):
        print(f"Soy un {self.__class__.__name__.lower()}")

In [13]:
# Usando la interfaz

class Coche(Vehiculo):
    def arrancar(self):
        print("El coche está arrancando")

    def detener(self):
        print("El coche se ha detenido")

class Bicicleta(Vehiculo):
    def arrancar(self):
        print("La bicicleta está en movimiento")

    def detener(self):
        print("La bicicleta se ha detenido")

In [15]:
coche = Coche()
coche.arrancar()
coche.detener()
coche.identificar()

bicicleta = Bicicleta()
bicicleta.arrancar()
bicicleta.detener()
bicicleta.identificar()

El coche está arrancando
El coche se ha detenido
Soy un coche
La bicicleta está en movimiento
La bicicleta se ha detenido
Soy un bicicleta


## **S**ingle responsability (Responsabilidad única)

Un componente debería tener una única responsabilidad o propósito.

In [ ]:
# Incumpliendo el principio
import datetime

def conexion_base_datos(url, usuario, contraseña, *, time=24*60*60):
    def validar(usuario, contraseña) -> bool:
        ...
        return True

    def hacer_cosas_en_base_datos():
        ...

    exito_conexion = True if mysql.connector.connect(host=url,database='mi_base_datos') else False
    if exito_conexion:
        print("Conexion exitosa")
    else:
        raise ValueError("La url no es valida")  # Se tendria una excepcion propia
    exito_validacion = validar(usuario, contraseña)
    if exito_validacion:
        ahora = datetime.datetime.now()
        limite_tiempo = ahora + datetime.timedelta(seconds=time)
        while limite_tiempo > ahora:
            ahora = datetime.datetime.now()
            hacer_cosas_en_base_datos()

In [ ]:
# Cumpliendo el principio

import datetime

def conectar(url) -> bool:
        ...
        return True

def validar(usuario, contraseña) -> bool:
        ...
        return True

def hacer_cosas_en_base_datos():
        ...

def mantener_conexion(time=24*60*60):
    ahora = datetime.datetime.now()
    limite_tiempo = ahora + datetime.timedelta(seconds=time)
    while limite_tiempo > ahora:
        ahora = datetime.datetime.now()
        hacer_cosas_en_base_datos()

def conexion_base_datos(url, usuario, contraseña, *, time=24*60*60):
    exito_conexion = conectar(url)
    if exito_conexion:
        print("Conexion exitosa")
    else:
        raise ValueError("La url no es valida")  # Se tendria una excepcion propia
    exito_validacion = validar(usuario, contraseña)
    if exito_validacion:
        mantener_conexion()


## **O**pen-Closed (Abierto a extensión, cerrado a modificación)

El software debe estar *abierto para extensión, pero cerrado para modificación*. Esto significa que deberías poder agregar nuevas funcionalidades sin cambiar el código existente.

In [ ]:
# Incumpliendo el principio

class Calculadora:
    def sumar(a, b):
        return a + b

    def restar(a, b):
        return a - b

    def cuadrado(a):  # MAL!!!
        return a ** 2

In [ ]:
# Cumpliendo el principio

class Calculadora:
    def sumar(a, b):
        return a + b

    def restar(a, b):
        return a - b

    def multiplicar(a, b)
        return a * b

class CalculadoraCientifica(Calculadora):
    def cuadrado(a):  # BIEN!!!
        return a ** 2

## **L**iskov substitution (Sustitución de Liskov)

Los objetos de una *clase derivada* **deben poder sustituir** a los objetos de una *clase padre* **sin alterar el comportamiento**.

In [ ]:
# Incumpliendo el principio

class Pajaro(ABC):
    def volar(self):
        print("Esta volando")

    @abstractmethod
    def comer_grano(self):
        ...

class Paloma(Pajaro):
    def comer_grano(self):
        print("La paloma come un poco de maiz")

class Pinguino(Pajaro):
    def volar():
        print("Este pajaro no vuela")  # MAL!!!
    
    def comer_grano(self):
        raise Exception("Los pingüinos no pueden comer grano!")

In [ ]:
# Cumpliendo el principio

class Pajaro(ABC):
    alas = True
    
    @abstractmethod
    def comer(self):
        ...

class Paloma(Pajaro):
    def comer(self):
        print("La paloma come un poco de maiz")

    def volar(self):
        print("La paloma esta volando")

class Pinguino(Pajaro):
    def comer(self):
        print("El pinguino come un pescado")

## **I**nterface segregation (Segregación de interfaces)

No se deberían de utilizar dependencias de interfaces que no usan. Esto significa que es mejor tener interfaces más específicas y pequeñas en lugar de una única interfaz grande.

In [ ]:
# Incumpliendo el principio

class Animal(ABC):
    def comer(self):
        print("El animal esta comiendo")

    @abstractmethod
    def correr(self):
        pass

    @abstractmethod
    def volar(self):
        pass

    @abstractmethod
    def nadar(self):
        pass


class Caballo(Animal):
    def correr(self):
        print("El caballo esta corriendo")

class Pinguino(Animal):
    def nadar(self):
        print("El pingüino esta corriendo")

    def correr(self):
        print("El pingüino esta corriendo")

class Mirlo(Animal):
    def correr(self):
        print("El mirlo esta corriendo")

    def volar(self):
        print("El mirlo esta volando")


In [ ]:
# Cumpliendo el principio

class Animal(ABC):
    def comer(self):
        print("El animal esta comiendo")

class Volador(ABC):
    @abstractmethod
    def volar(self):
        pass

class Acuatico(ABC):
    @abstractmethod
    def nadar(self):
        pass

class Terrestre(ABC):
    @abstractmethod
    def correr(self):
        pass



class Caballo(Animal, Terrestre):
    def correr(self):
        print("El caballo esta corriendo")

class Pinguino(Animal, Terrestre, Acuatico):
    def nadar(self):
        print("El pingüino esta corriendo")

    def correr(self):
        print("El pingüino esta corriendo")

class Mirlo(Animal, Terrestre, Volador):
    def correr(self):
        print("El mirlo esta corriendo")

    def volar(self):
        print("El mirlo esta volando")


## **D**ependency inversion (Inversión de dependencias)

Las entidades de **alto nivel no deberían depender de entidades de bajo nivel**. Ambas deberían depender de **abstracciones**. Es decir. las abstracciones no deberían depender de los detalles, los detalles deberían depender de las abstracciones. Esto se consigue mediante **inyección de dependencias**.

Este posiblemente sea el principio más importante y más complicado de aplicar. Un buen uso de el hace que el código este bien desacoplado y se mantenga facil de mantener y cambiar.

In [ ]:
# Incumpliendo el principio

class Bombilla:
    def on(self):
        print("Se ha encendido")

    def off(self):
        print("Se ha apagado")

class Interruptor:
    def __init__(self):
        self.status = False  # La bombilla empieza apagada

    def on_off(self):
        if self.status:
            self.status = False
            self.bombilla.off()
        else:
            self.status = True
            self.bombilla.on()

# Implementar ahora un ventilador!

In [ ]:
# Cumpliendo el principio

from abc import ABC

class Encendible(ABC):
    def on(self):
        print(f"Encendido {self.__class__.__name__}")

    def off(self):
        print(f"Apagado {self.__class__.__name__}")

class Bombilla(Encendible):
    def __init__(self):
        self.status = False
        
class Ventilador(Encendible):
    def __init__(self):
        self.status = False

class Interruptor:
    def __init__(self, aparato: Encendible):
        self.aparato = aparato

    def on_off(self):
        if self.aparato.status:
            self.aparato.off()
            self.aparato.status = False
        else:
            self.aparato.on()
            self.aparato.status = True
            
bombilla = Bombilla()
ventilador = Ventilador()
circuito = Interruptor(bombilla)
circuito.on_off()  # Encendida
circuito.on_off()  # Apagada
circuito = Interruptor(ventilador)
circuito.on_off()
circuito.on_off()
circuito.on_off()
print(bombilla.status)
print(ventilador.status)


In [ ]:
# Aplicar los 5 principios para hacer un sistema de notificaciones por SMS, Email, Whatsapp, Telegram y un sistema interno por Push